In [ ]:
import sys
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats
#sys.path
import scipy





import datetime
import math
import time
import itertools


from pymongo import MongoClient

import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff


########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [ ]:
label_journal =  'PRE'  #  SCIENCE  NATURE   PNAS   PRL   PRE  JTB



  
    
    
#### this values must match the ones used in the "queries_PRL_Nature_Science_PNAS_etc.ipynb"  notebook, and they are only to read automatically the files generated there
lista_years =[1990,1991,1992,1993,1994,1995]                      
    
    
value_top = 1000            #   SAME VALUE FOR PNAS AND PRL  (1000),   SAME VALUE FOR NATURE AND SCIENCE (2000)    
interval_intermediate=[95,105]    
interval_low=[10,20]    

size_random_sample_papers = 500
if label_journal == 'NATURE'  or label_journal == 'SCIENCE':
    value_top = 2000   
    size_random_sample_papers = 200

elif label_journal == 'PNAS':
    value_top = 1000 
    
    
elif label_journal == 'PRL':
    value_top = 1000   
    size_random_sample_papers = 500

    
elif label_journal == 'PRE':
    value_top = 250    
    interval_intermediate=[45,55]    
    interval_low=[1,4]    
    
    
elif label_journal == 'JTB':
    value_top = 170    
    interval_intermediate=[55,75]    
    interval_low=[5,15]    
    size_random_sample_papers = 200
    
   
   
    
    
   
    
    
    


dict_UT_ref_of_selected_top_JOURNAL_info = pickle.load(open('../data/dict_citing_UT_of_selected_UT_'+label_journal+'_top_info.pkl', 'rb'))
dict_UT_ref_of_selected_intermediate_JOURNAL_info = pickle.load(open('../data/dict_citing_UT_of_selected_UT_'+label_journal+'_intermediate_info.pkl', 'rb'))
dict_UT_ref_of_selected_low_JOURNAL_info = pickle.load(open('../data/dict_citing_UT_of_selected_UT_'+label_journal+'_low_info.pkl', 'rb'))
dict_UT_ref_of_selected_RANDOM_SAMPLE_JOURNAL_info = pickle.load(open('../data/dict_citing_UT_of_selected_UT_'+label_journal+"_RANDOM_SAMPLE"+str(size_random_sample_papers)+'_info.pkl', 'rb'))                                                                        



lista_dicts = [dict_UT_ref_of_selected_top_JOURNAL_info,dict_UT_ref_of_selected_intermediate_JOURNAL_info, dict_UT_ref_of_selected_low_JOURNAL_info,\
              dict_UT_ref_of_selected_RANDOM_SAMPLE_JOURNAL_info]




list_top_UTs = pickle.load(open('../data/list_top_UTs'+str(lista_years[0])+'_'+str(lista_years[-1])+'_'+label_journal+'_'+str(value_top)+'.pkl', 'rb'))
list_intermediate_UTs = pickle.load(open('../data/list_intermediate_UTs'+str(lista_years[0])+'_'+str(lista_years[-1])+'_'+label_journal+'_'+str(interval_intermediate[0])+'_'+str(interval_intermediate[-1])+'.pkl', 'rb'))
list_low_UTs = pickle.load(open('../data/list_low_UTs'+str(lista_years[0])+'_'+str(lista_years[-1])+'_'+label_journal+'_'+str(interval_low[0])+'_'+str(interval_low[-1])+'.pkl', 'rb'))
list_RANDOM_SAMPLE_UTs = pickle.load(open('../data/list_RANDOM_UTs'+str(lista_years[0])+'_'+str(lista_years[-1])+'_'+label_journal+'.pkl', 'rb'))

lista_listas_to_look_for = [list_top_UTs, list_intermediate_UTs, list_low_UTs, list_RANDOM_SAMPLE_UTs] 
    

master_dict_JOURNAL_UT_info  =  pickle.load(open('../data/dict_'+label_journal+'_UT_info_year'+str(lista_years[0])+'_'+str(lista_years[-1])+'_any_num_cit.pkl', 'rb'))

    
print ('done.')

In [ ]:
len(master_dict_JOURNAL_UT_info)

In [ ]:
### box plots with offset for the different traces, so it is easy to see

data = []


include_random = 'no'

label_plot_yplus1 = 'yes'  #'yes'



label_log_y = 'log10'  # or log or log10  or no  (ojo! only active if if label_plot_yplus1=1)  ## also comment lines in the layout about range!!



initial_offset = -0.3
delta_offset_value = 0.15   # this will also afect the width of the boxes



range_y = [0.24,2.51]


# this is for plotting number of citations (not log of numb cit)
# if label_journal == "PRL":
#     range_y = [.35,3.3 ]
    
# elif label_journal == "PRE":
#     range_y = [.2,3.3 ]
    
# elif label_journal == "PNAS":
#     range_y = [.25,3.3 ]



 

print (label_journal)
    
# lista_tickvals_y = [2,5,10,20,50,100,200,500,1000 ]    
lista_tickvals_y = [.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75]   # this is just the labels of the axis, not the range of values themselves!
# if label_log_y != 'no' :
#     lista_tickvals_y = [0,1,2,3,4,5,6,7,8,9 ]    



boxpoints_option = False  #'outliers'   # 'outliers',  #'suspectedoutliers',  #False, #'all',  # False,# THIS DICTATES THE BEHAVIOUR OF THE WHISKERS!
opacity_points = 0.     # to hide all points but keep the same whiskers, set it to 0.0
opacity_box = 1.    



width_line = 7
width_whisker = 1.     # [0,1] value  (horizontal line of whisker)
width_line_whiskers = 4.  # vertical line of whisker
    
    
    
#### for the colors of top, intermediate and low

color_rand =  '#af8dc3'   #  purple    lista_colors[-1]



if label_journal == "PNAS":
    lista_colores = ['#330080','#6600ff','#c299ff']  # purples 
    color_rand = '#006d2c'
    
elif label_journal == "PRE":
    lista_colores =  ['#804000','#ff8c1a','#ffe680']    #    ['#993404','#fe9929','#fee391']   # oranges
    
elif label_journal == "PRL":
    lista_colores = ['#08519c', '#6baed6','#c6dbef']   ## blues

elif label_journal == "SCIENCE":
    lista_colores = ['#a50f15','#fb6a4a','#fcbba1']   # reds        
        
elif label_journal == "NATURE":                 
     lista_colores = ['#006d2c','#66c2a4','#ccece6']      ## greens            

elif label_journal == "JTB":
    lista_colores = ['#252525','#636363','#bdbdbd']  # greys




            
title_y='Number of citations'
if label_plot_yplus1 =='yes' :            
    title_y='Number of citations +1'
    if label_log_y == 'log'  :
        
        title_y='Ln(Number of citations +1)'
    if label_log_y == 'log10'  :
        
        title_y='log10(Number of citations +1)'        
            
            
font_gral = 60






#################
#### TRACE:   THE TOP  values


label_selected = 't'
if label_selected == 't':  
    dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_top_JOURNAL_info 
    lista_JOURNAL_to_look_for = list_top_UTs
    
    offset = initial_offset#-2.*offset_value


   ### get the data
master_dict_reset_year_list_citations_of_citing_papers = {}
for JOURNAL_UT in lista_JOURNAL_to_look_for:   # either i focus on top JOURNAL papers, intermediate or low     

    JOURNAL_year = master_dict_JOURNAL_UT_info[JOURNAL_UT]['year']
    
    list_citing_papers_of_JOURNAL = master_dict_JOURNAL_UT_info[JOURNAL_UT]['list_cit']


    for citing_paper in list_citing_papers_of_JOURNAL:

        try:
            year = dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['year']
            num_citations_of_citing = len(dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['list_cit'])

            reset_year = year - JOURNAL_year

            try:
                master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)
            except KeyError:
                master_dict_reset_year_list_citations_of_citing_papers[reset_year] = []
                master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)

        except: pass

        

cont  = 0        
lista_val_line_x = []
lista_val_line_y = []
for reset_year in sorted(master_dict_reset_year_list_citations_of_citing_papers):
    if (reset_year <=10)  and (reset_year >= 0):   ### i only plot the first 10 years
#         print (reset_year, np.median(master_dict_reset_year_list_citations_of_citing_papers[reset_year]),np.mean(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), \
#                2.*np.std(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), len(set(master_dict_reset_year_list_citations_of_citing_papers[reset_year])))

        
       
        lista_y = master_dict_reset_year_list_citations_of_citing_papers[reset_year]
        if label_plot_yplus1 =='yes':
            lista_y = [i+1 for i in master_dict_reset_year_list_citations_of_citing_papers[reset_year]]
            
            if label_log_y == 'log':
                lista_y = [np.log(i) for i in lista_y]
            elif label_log_y == 'log10':
                lista_y = [np.log10(i) for i in lista_y]
        
        
        lista_x = [(reset_year+offset)]* len(lista_y)
        
        
        
        lista_val_line_y.append(np.median(lista_y))
        lista_val_line_x.append(reset_year+offset)
        
        
        name = ''
        flag_show_legend = False
        if cont ==0:
            flag_show_legend = True
            name = '>'+str(value_top)+" citations"
        
        
        
        trace0 = go.Box(
                y = lista_y,  
                x = lista_x, 
                name = name,
                opacity = opacity_box, 
                whiskerwidth= 0,#width_whisker ,
                marker = dict(
                            color = lista_colores[0],#color,#'##size = 0,  # this doesnt work!!!
                            #opacity = opacity_points, 
                        ),
                boxmean=True,
                showlegend=flag_show_legend,
                #boxpoints = False,  #boxpoints_option, #'outliers',  #'suspectedoutliers',  #False, #'all',  # False,#
                line=dict(width=0)#width_line_whiskers),
            )
        
       
         
                
                
        
        cont  +=1
        

        data.append(trace0)    

    
    
    
trace1 = go.Scatter(
    x = lista_val_line_x,
    y = lista_val_line_y,
    mode = 'lines+markers',    
    line = dict(
        color = lista_colores[0],#color,
        width = width_line,),
    showlegend=False
    
    
    
)    
data.append(trace1)
        
# print (offset)
        

    
## i fit to an exponential decay:
#  x = numpy.array([10, 19, 30, 35, 51])
# >>> y = numpy.array([1, 7, 20, 50, 79])

# scipy.optimize.curve_fit(lambda t,a,b: a*numpy.exp(b*t),  x,  y,  p0=(4, 0.1))  #  p0 is the initial guess for the papram
# (array([ 4.88003249,  0.05531256]),
#  array([[  1.01261314e+01,  -4.31940132e-02],
#         [ -4.31940132e-02,   1.91188656e-04]]))   
# y ≈ 4.88 exp(0.0553 x). much better.




print ("fit to trace: highly-cited")
#results = scipy.optimize.curve_fit(lambda t,A,landa: A*np.exp(-1.*landa*t),  lista_val_line_x,  lista_val_line_y)
# print (results)


results = scipy.optimize.curve_fit(lambda t,A,landa: A*np.exp(-1.*landa*t),  lista_val_line_x,  lista_val_line_y, p0=(lista_val_line_y[0],0.1))
# print (results)


A = results[0][0]
landa = results[0][1]
tau = 1./landa
half_life = tau* np.log(2) 
    
print ("A:",A,",   landa:", landa,',   tau:', round(tau,1),"years,     half-life:",round(half_life,1), " years\n")


#τ is the time at which the population of the assembly is reduced to 1/e ≈ 0.367879441 times its initial value.
### interpretaion of example results:
# (array([ 4.88003249,  0.05531256]),
#  array([[  1.01261314e+01,  -4.31940132e-02],
#         [ -4.31940132e-02,   1.91188656e-04]]))
# # y ≈ 4.88 exp(0.0553 x). much better.
        
  



        

#################
#### TRACE:   THE INTERMEDIATE  values


label_selected = 'i'
if label_selected == 'i':  
    dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_intermediate_JOURNAL_info 
    lista_JOURNAL_to_look_for = list_intermediate_UTs
   
    offset += delta_offset_value 


   ### get the data
master_dict_reset_year_list_citations_of_citing_papers = {}

for JOURNAL_UT in lista_JOURNAL_to_look_for:   # either i focus on top JOURNAL papers, intermediate or low     

    JOURNAL_year = master_dict_JOURNAL_UT_info[JOURNAL_UT]['year']
    
    list_citing_papers_of_JOURNAL = master_dict_JOURNAL_UT_info[JOURNAL_UT]['list_cit']


    for citing_paper in list_citing_papers_of_JOURNAL:

        try:
            year = dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['year']
            num_citations_of_citing = len(dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['list_cit'])

            reset_year = year - JOURNAL_year

            try:
                master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)
            except KeyError:
                master_dict_reset_year_list_citations_of_citing_papers[reset_year] = []
                master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)

        except: pass


        
cont = 0
lista_val_line_y = []
lista_val_line_x = []        
for reset_year in sorted(master_dict_reset_year_list_citations_of_citing_papers):
    if (reset_year <=10)  and (reset_year >= 0):   ### i only plot the first 10 years
#         print (reset_year, np.median(master_dict_reset_year_list_citations_of_citing_papers[reset_year]),np.mean(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), \
#                2.*np.std(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), len(set(master_dict_reset_year_list_citations_of_citing_papers[reset_year])))

        
        lista_y = master_dict_reset_year_list_citations_of_citing_papers[reset_year]
        if label_plot_yplus1 =='yes':
            lista_y = [i+1 for i in master_dict_reset_year_list_citations_of_citing_papers[reset_year]]
            
            if label_log_y == 'log':
                lista_y = [np.log(i) for i in lista_y]
            elif label_log_y == 'log10':
                lista_y = [np.log10(i) for i in lista_y]        
                
                
                
        
        lista_x = [(reset_year+offset)]* len(lista_y)

        
        lista_val_line_y.append(np.median(lista_y))
        lista_val_line_x.append(reset_year+offset)
        
        name = ''
        flag_show_legend = False
        if cont ==0:
            flag_show_legend = True
            name = str(interval_intermediate[0])+'-'+str(interval_intermediate[-1])+" citations"
        
            
            
        trace0 = go.Box(
                y = lista_y,  
                x = lista_x,  
                name = name,
                opacity = opacity_box, 
                whiskerwidth= width_whisker ,
                marker = dict(
                            color = lista_colores[1],#color,#'##size = 0,  # this doesnt work!!!
                            opacity = opacity_points, 
                        ),
                boxmean=True,                
                 showlegend=flag_show_legend,
                #boxpoints = False,  #boxpoints_option, #'outliers',  #'suspectedoutliers',  #False, #'all',  # False,#
                line=dict(width=0)#width_line_whiskers),
            )

        
        
        
        cont  +=1
        
        data.append(trace0)

    
        
#   value_top = 170    
#     interval_intermediate=[55,75]    
#     interval_low=[5,15] 

    
    
trace1 = go.Scatter(
    x = lista_val_line_x,
    y = lista_val_line_y,
    mode = 'lines+markers',    
    line = dict(
        color = lista_colores[1],#color,
        width = width_line,),
    showlegend=False,        
    
)    
data.append(trace1)


#print (offset)      

        
        
print ("fit to trace: medium-cited")
#results = scipy.optimize.curve_fit(lambda t,A,landa: A*np.exp(-1.*landa*t),  lista_val_line_x,  lista_val_line_y)
# print (results)


results = scipy.optimize.curve_fit(lambda t,A,landa: A*np.exp(-1.*landa*t),  lista_val_line_x,  lista_val_line_y, p0=(lista_val_line_y[0],0.1))
# print (results)


A = results[0][0]
landa = results[0][1]
tau = 1./landa
half_life = tau* np.log(2) 
    
print ("A:",A,",   landa:", landa,',   tau:', round(tau,1),"years,     half-life:",round(half_life,1), " years\n")





#################
#### TRACE:   THE LOW  values

   
label_selected = 'l'
if label_selected == 'l':  
   
    dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_low_JOURNAL_info 
    lista_JOURNAL_to_look_for = list_low_UTs
 
           
    offset += delta_offset_value 


   ### get the data
master_dict_reset_year_list_citations_of_citing_papers = {}

for JOURNAL_UT in lista_JOURNAL_to_look_for:   # either i focus on top JOURNAL papers, intermediate or low     

    JOURNAL_year = master_dict_JOURNAL_UT_info[JOURNAL_UT]['year']
    
    list_citing_papers_of_JOURNAL = master_dict_JOURNAL_UT_info[JOURNAL_UT]['list_cit']


    for citing_paper in list_citing_papers_of_JOURNAL:

        try:
            year = dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['year']
            num_citations_of_citing = len(dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['list_cit'])

            reset_year = year - JOURNAL_year

            try:
                master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)
            except KeyError:
                master_dict_reset_year_list_citations_of_citing_papers[reset_year] = []
                master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)

        except: pass


        

        
cont = 0
lista_val_line_y = []
lista_val_line_x = []
for reset_year in sorted(master_dict_reset_year_list_citations_of_citing_papers):
    if (reset_year <=10)  and (reset_year >= 0):   ### i only plot the first 10 years
#         print (reset_year, np.median(master_dict_reset_year_list_citations_of_citing_papers[reset_year]),np.mean(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), \
#                2.*np.std(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), len(set(master_dict_reset_year_list_citations_of_citing_papers[reset_year])))

        
        lista_y = master_dict_reset_year_list_citations_of_citing_papers[reset_year]
        if label_plot_yplus1 =='yes':
            lista_y = [i+1 for i in master_dict_reset_year_list_citations_of_citing_papers[reset_year]]
            
            if label_log_y == 'log':
                lista_y = [np.log(i) for i in lista_y]
            elif label_log_y == 'log10':
                lista_y = [np.log10(i) for i in lista_y]        
       
    
        
        lista_x = [(reset_year+offset)]* len(lista_y)
        
        lista_val_line_y.append(np.median(lista_y))
        lista_val_line_x.append(reset_year+offset)

        
        name = ''
        flag_show_legend = False
        if cont ==0:
            flag_show_legend = True
            name = str(interval_low[0])+'-'+str(interval_low[-1])+" citations"
        
        
        trace0 = go.Box(
                 y = lista_y,  
                x = lista_x,    
                name = name ,
                opacity = opacity_box, 
                whiskerwidth= width_whisker ,
                marker = dict(
                            color = lista_colores[2],#color,#'##size = 0,  # this doesnt work!!!
                            opacity = opacity_points, 
                        ),
                boxmean=True,                
                showlegend=flag_show_legend,
                #boxpoints = False,  #boxpoints_option, #'outliers',  #'suspectedoutliers',  #False, #'all',  # False,#
                line=dict(width=0)#width_line_whiskers),
          
            )

        
        
        
        cont  +=1
        
        data.append(trace0)



    
    
trace1 = go.Scatter(
    x = lista_val_line_x,
    y = lista_val_line_y,
    mode = 'lines+markers',    
    line = dict(
        color = lista_colores[2],#color,
        width = width_line,),
    showlegend=False,
  
)    
data.append(trace1)

# print (offset)        

        
print ("fit to trace: poorly-cited")
#results = scipy.optimize.curve_fit(lambda t,A,landa: A*np.exp(-1.*landa*t),  lista_val_line_x,  lista_val_line_y)
# print (results)


results = scipy.optimize.curve_fit(lambda t,A,landa: A*np.exp(-1.*landa*t),  lista_val_line_x,  lista_val_line_y, p0=(lista_val_line_y[0],0.1))
# print (results)


A = results[0][0]
landa = results[0][1]
tau = 1./landa
half_life = tau* np.log(2) 
    
print ("A:",A,",   landa:", landa,',   tau:', round(tau,1),"years,     half-life:",round(half_life,1), " years\n")


        
        
 
     

#################
#### TRACE:   THE RANDOM DATA

if include_random == 'yes':

    label_selected = 'r'
    if label_selected == 'r':  
        dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_RANDOM_SAMPLE_JOURNAL_info 
        lista_JOURNAL_to_look_for = list_RANDOM_SAMPLE_UTs

        
        offset += delta_offset_value 


       ### get the data
    master_dict_reset_year_list_citations_of_citing_papers = {}
    for JOURNAL_UT in lista_JOURNAL_to_look_for:   # either i focus on top JOURNAL papers, intermediate or low     

        JOURNAL_year = master_dict_JOURNAL_UT_info[JOURNAL_UT]['year']
        
        list_citing_papers_of_JOURNAL = master_dict_JOURNAL_UT_info[JOURNAL_UT]['list_cit']


        for citing_paper in list_citing_papers_of_JOURNAL:

            try:
                year = dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['year']
                num_citations_of_citing = len(dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['list_cit'])

                reset_year = year - JOURNAL_year

                try:
                    master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)
                except KeyError:
                    master_dict_reset_year_list_citations_of_citing_papers[reset_year] = []
                    master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)

            except: pass


    cont = 0
    lista_val_line_y = []
    lista_val_line_x = []        

    for reset_year in sorted(master_dict_reset_year_list_citations_of_citing_papers):
        if (reset_year <=10)  and (reset_year >= 0):   ### i only plot the first 10 years
#             print (reset_year, np.median(master_dict_reset_year_list_citations_of_citing_papers[reset_year]),np.mean(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), \
#                    2.*np.std(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), len(set(master_dict_reset_year_list_citations_of_citing_papers[reset_year])))

            
            lista_y = master_dict_reset_year_list_citations_of_citing_papers[reset_year]
            if label_plot_yplus1 =='yes':
                lista_y = [i+1 for i in master_dict_reset_year_list_citations_of_citing_papers[reset_year]]
                
            if label_log_y == 'log':
                lista_y = [np.log(i) for i in lista_y]
            elif label_log_y == 'log10':
                lista_y = [np.log10(i) for i in lista_y]        
       
                
                
            lista_x = [(reset_year+offset)]* len(lista_y)

            lista_val_line_y.append(np.median(lista_y))
            lista_val_line_x.append(reset_year+offset)

            flag_show_legend = False
            if cont ==0:
                 flag_show_legend = True
        

            trace0 = go.Box(
                    y = lista_y,  
                    x = lista_x,                
                    opacity = opacity_box, 
                    whiskerwidth= width_whisker ,
                    marker = dict(
                                color = color_rand,#color,#'##size = 0,  # this doesnt work!!!
                                opacity = opacity_points, 
                            ),
                    boxmean=True,
                    showlegend=False,
                    boxpoints = boxpoints_option, #'outliers',  #'suspectedoutliers',  #False, #'all',  # False,#
                    line=dict(width=width_line_whiskers),
                )


           
            cont  +=1



            data.append(trace0)

            



    trace1 = go.Scatter(
        x = lista_val_line_x,
        y = lista_val_line_y,
        mode = 'lines+markers',    
        line = dict(
            color = color_rand,
            width = width_line,),
        showlegend=False,


    )    
    data.append(trace1)


   # print (offset)


        
           
      
        
        
        
        
            
    
    
    
    
    
   
layout = Layout(    
    title = '',#citing papers of '+label_selected+' JOURNALJOURNALfo+"<br>N:"+n,
    # showlegend=True,  IF I WRITE THIS INSIDE EACH TRACE, I PICK WHICH LENGENDS TO SHOW!
    legend=dict(x=.7, 
                y=.95,
               font=dict(            
                    size=font_gral-5,           
                ),
               ),
    yaxis=dict(
        title=title_y, 
        range = range_y,      ###### un-comment this line if i want to plot linear y-axis!!!
       #type='log',           ###### un-comment this line if i want to plot linear y-axis!!!
        titlefont=dict(            
            size=font_gral,
            color='black'
        #    color='lightgrey'
        ),  
        tickfont=dict(               
            size=font_gral-15,
            color='black'
        ),
        tickvals = lista_tickvals_y ,
        showgrid=True,
        gridcolor='#525252',
        gridwidth=1,
    ),  
       
    
    
    xaxis=dict(
        title='Age difference', 
        titlefont=dict(            
            size=font_gral,
            color='black'
        #    color='lightgrey'
        ),  
        tickfont=dict(               
            size=font_gral -15, 
            color='black'
        ),
        tickvals = [0,1,2,3,4,5,6,7,8,9,10],
    ),
    
    
    
    
        margin=go.Margin(
        l=300,
       # r=50,
        b=120,
        t=20,
        #pad=80
    ),

    boxgap=0.01
    
    
    
    

)
    
    
fig = Figure(data, layout =  layout)

fig_name='box_plot_offset_'+label_journal
path='/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/'


offline.plot(fig, auto_open=True, image = 'png', image_filename=fig_name ,
                 output_type='file',image_width=2200, image_height=1800, filename=path+fig_name+".html", validate=False)



#### PNAS
# fit to trace: highly-cited
# A: 79.5775433126 ,   landa: 0.127033119861 ,   tau: 7.9 years,     half-life: 5.5  years

# fit to trace: medium-cited
# A: 52.3421128686 ,   landa: 0.0857932567604 ,   tau: 11.7 years,     half-life: 8.1  years

# fit to trace: poorly-cited
# A: 23.1535757648 ,   landa: 0.0282811138629 ,   tau: 35.4 years,     half-life: 24.5  years




# SCIENCE
# fit to trace: highly-cited
# A: 73.5566847391 ,   landa: 0.14482791378 ,   tau: 6.9 years,     half-life: 4.8  years

# fit to trace: medium-cited
# A: 41.880926933 ,   landa: 0.0645084725871 ,   tau: 15.5 years,     half-life: 10.7  years

# fit to trace: poorly-cited
# A: 19.5927563684 ,   landa: -0.000444452670601 ,   tau: -2250.0 years,     half-life: -1559.6  years




# NATURE
# fit to trace: highly-cited
# A: 59.5875784412 ,   landa: 0.0875669423552 ,   tau: 11.4 years,     half-life: 7.9  years

# fit to trace: medium-cited
# A: 41.6401832307 ,   landa: 0.0480770365393 ,   tau: 20.8 years,     half-life: 14.4  years

# fit to trace: poorly-cited
# A: 21.0361533805 ,   landa: 0.00208731952169 ,   tau: 479.1 years,     half-life: 332.1  years


# PRE
# fit to trace: highly-cited
# A: 42.484635282 ,   landa: 0.104757926239 ,   tau: 9.5 years,     half-life: 6.6  years

# fit to trace: medium-cited
# A: 23.7676029139 ,   landa: 0.0834063429733 ,   tau: 12.0 years,     half-life: 8.3  years

# fit to trace: poorly-cited
# A: 5.46116383289 ,   landa: -0.0350143554733 ,   tau: -28.6 years,     half-life: -19.8  years




# PRL
# fit to trace: highly-cited
# A: 60.1177575044 ,   landa: 0.292642091292 ,   tau: 3.4 years,     half-life: 2.4  years

# fit to trace: medium-cited
# A: 25.9369434312 ,   landa: 0.110985782474 ,   tau: 9.0 years,     half-life: 6.2  years

# fit to trace: poorly-cited
# A: 11.837182788 ,   landa: 0.0430482652405 ,   tau: 23.2 years,     half-life: 16.1  years



# JTB
# fit to trace: highly-cited
# A: 36.0663881783 ,   landa: -0.0129775250214 ,   tau: -77.1 years,     half-life: -53.4  years

# fit to trace: medium-cited
# A: 28.7488070266 ,   landa: 0.0149116465487 ,   tau: 67.1 years,     half-life: 46.5  years

# fit to trace: poorly-cited
# A: 13.0394080734 ,   landa: -0.0284148992609 ,   tau: -35.2 years,     half-life: -24.4  years


In [ ]:
### running KS test between pairs of distributions:
for year in range(11):

    for i in range (11):
        if i >= year:
            lista1 = master_dict_reset_year_list_citations_of_citing_papers[year]
            lista2 = master_dict_reset_year_list_citations_of_citing_papers[i]

        #     new_lista1 =  [np.log(item) for item in lista1]
        #     new_lista2 =  [np.log(item) for item in lista2]


            print ("year",year, ' vs ',i, ':',stats.ks_2samp(lista1,lista2 )[1])
            # If the K-S statistic is small or the p-value is high, then we cannot reject the hypothesis that the distributions of the two samples are the same.
    print (" ")

In [ ]:
# # #  p-value of KS test 2-sample PNAS:  higly cited >1000 citation group: 

# year 0  vs  0 : 1.0
# year 0  vs  1 : 0.000210917513462
# year 0  vs  2 : 1.05582830755e-07
# year 0  vs  3 : 3.71520743252e-13
# year 0  vs  4 : 1.03645778229e-15
# year 0  vs  5 : 1.73829823336e-19
# year 0  vs  6 : 1.83609492139e-21
# year 0  vs  7 : 3.12114316708e-23
# year 0  vs  8 : 5.84619288202e-25
# year 0  vs  9 : 9.79069887555e-28
# year 0  vs  10 : 3.93470346264e-29
 
# year 1  vs  1 : 1.0
# year 1  vs  2 : 0.000158299312426
# year 1  vs  3 : 1.29701471806e-22
# year 1  vs  4 : 1.97770571732e-38
# year 1  vs  5 : 1.03138839049e-62
# year 1  vs  6 : 3.27068773788e-73
# year 1  vs  7 : 7.80545764259e-87
# year 1  vs  8 : 2.06661134251e-86
# year 1  vs  9 : 3.48920492618e-102
# year 1  vs  10 : 1.24863780558e-113
 
# year 2  vs  2 : 1.0
# year 2  vs  3 : 3.25924567916e-12
# year 2  vs  4 : 1.48561343278e-28
# year 2  vs  5 : 7.03419140466e-56
# year 2  vs  6 : 4.09172338702e-68
# year 2  vs  7 : 9.74690974768e-85
# year 2  vs  8 : 1.53407623321e-83
# year 2  vs  9 : 5.13659279589e-103
# year 2  vs  10 : 1.05115221068e-117
 
# year 3  vs  3 : 1.0
# year 3  vs  4 : 1.00120022725e-05
# year 3  vs  5 : 8.34676369215e-23
# year 3  vs  6 : 7.44348565292e-32
# year 3  vs  7 : 1.04575263071e-43
# year 3  vs  8 : 3.75994651178e-42
# year 3  vs  9 : 1.19875169438e-57
# year 3  vs  10 : 1.59685682594e-66
 
# year 4  vs  4 : 1.0
# year 4  vs  5 : 2.84560788828e-07
# year 4  vs  6 : 2.74269630683e-13
# year 4  vs  7 : 5.21014652143e-22
# year 4  vs  8 : 7.76746170027e-21
# year 4  vs  9 : 1.9017863499e-32
# year 4  vs  10 : 2.9430417224e-42
 
# year 5  vs  5 : 1.0
# year 5  vs  6 : 0.00611795450185
# year 5  vs  7 : 4.52871757923e-07
# year 5  vs  8 : 1.23691384453e-06
# year 5  vs  9 : 1.47378336628e-13
# year 5  vs  10 : 6.79173514922e-21
 
# year 6  vs  6 : 1.0
# year 6  vs  7 : 0.0323595549436
# year 6  vs  8 : 0.0081530813193
# year 6  vs  9 : 6.77596666376e-07
# year 6  vs  10 : 1.1121276272e-12
 
# year 7  vs  7 : 1.0
# year 7  vs  8 : 0.35072365099
# year 7  vs  9 : 0.00262698923883
# year 7  vs  10 : 7.41960596944e-06
 
# year 8  vs  8 : 1.0
# year 8  vs  9 : 0.0475819264305
# year 8  vs  10 : 1.96844116123e-05
 
# year 9  vs  9 : 1.0
# year 9  vs  10 : 0.155089817496
 
# year 10  vs  10 : 1.0
 


In [ ]:
# # #  p-value of KS test 2-sample PNAS:  mid-cited 95-105 citation group: 


# year 0  vs  0 : 1.0
# year 0  vs  1 : 0.000346657538114
# year 0  vs  2 : 6.62404878967e-11
# year 0  vs  3 : 1.1735372575e-17
# year 0  vs  4 : 7.13173537247e-22
# year 0  vs  5 : 2.52897968279e-27
# year 0  vs  6 : 2.33960216454e-31
# year 0  vs  7 : 9.72341361601e-35
# year 0  vs  8 : 2.1404083223e-37
# year 0  vs  9 : 8.18990172896e-38
# year 0  vs  10 : 4.63643582654e-41
 
# year 1  vs  1 : 1.0
# year 1  vs  2 : 5.2072604308e-09
# year 1  vs  3 : 1.27422317623e-21
# year 1  vs  4 : 1.02766593066e-30
# year 1  vs  5 : 3.66738861947e-45
# year 1  vs  6 : 2.43073957566e-53
# year 1  vs  7 : 1.189568019e-57
# year 1  vs  8 : 2.80484747483e-60
# year 1  vs  9 : 2.48527039364e-61
# year 1  vs  10 : 1.02672651051e-61
 
# year 2  vs  2 : 1.0
# year 2  vs  3 : 4.1479298197e-06
# year 2  vs  4 : 7.99384247418e-11
# year 2  vs  5 : 1.00598688623e-20
# year 2  vs  6 : 1.55940978237e-27
# year 2  vs  7 : 1.7130918772e-31
# year 2  vs  8 : 3.7989772632e-34
# year 2  vs  9 : 2.27693731105e-36
# year 2  vs  10 : 4.76961337454e-38
 
# year 3  vs  3 : 1.0
# year 3  vs  4 : 0.0238203933017
# year 3  vs  5 : 2.70286350838e-07
# year 3  vs  6 : 4.03873266956e-12
# year 3  vs  7 : 3.1035484211e-15
# year 3  vs  8 : 8.8689167124e-18
# year 3  vs  9 : 3.0544969574e-19
# year 3  vs  10 : 3.43934626601e-22
 
# year 4  vs  4 : 1.0
# year 4  vs  5 : 0.000673033759815
# year 4  vs  6 : 1.21721306642e-07
# year 4  vs  7 : 2.44614614584e-08
# year 4  vs  8 : 8.46894217783e-12
# year 4  vs  9 : 3.01157646003e-14
# year 4  vs  10 : 3.11955128165e-14
 
# year 5  vs  5 : 1.0
# year 5  vs  6 : 0.00589780550143
# year 5  vs  7 : 0.00303557473815
# year 5  vs  8 : 0.000258540315954
# year 5  vs  9 : 1.27567039179e-07
# year 5  vs  10 : 1.77525002758e-07
 
# year 6  vs  6 : 1.0
# year 6  vs  7 : 0.525025043231
# year 6  vs  8 : 0.0917953714075
# year 6  vs  9 : 0.00859236780187
# year 6  vs  10 : 0.000447849707589
 
# year 7  vs  7 : 1.0
# year 7  vs  8 : 0.639813192153
# year 7  vs  9 : 0.0134988869785
# year 7  vs  10 : 0.0386136971503
 
# year 8  vs  8 : 1.0
# year 8  vs  9 : 0.0706528698986
# year 8  vs  10 : 0.284923308628
 
# year 9  vs  9 : 1.0
# year 9  vs  10 : 0.410735357193
 
# year 10  vs  10 : 1.0

















In [ ]:
# # # #  p-value of KS test 2-sample PNAS:  poorly-cited 10-20 citation group: 

# year 0  vs  0 : 1.0
# year 0  vs  1 : 0.321910833743
# year 0  vs  2 : 0.800600001524
# year 0  vs  3 : 0.443181313465
# year 0  vs  4 : 0.40315511614
# year 0  vs  5 : 0.0230638992049
# year 0  vs  6 : 0.549012462462
# year 0  vs  7 : 0.0959430636799
# year 0  vs  8 : 0.613172478029
# year 0  vs  9 : 0.185408178949
# year 0  vs  10 : 0.0521202761056
 
# year 1  vs  1 : 1.0
# year 1  vs  2 : 0.238447923276
# year 1  vs  3 : 0.00284872228279
# year 1  vs  4 : 0.00187736860762
# year 1  vs  5 : 2.49839968703e-06
# year 1  vs  6 : 0.00540240869484
# year 1  vs  7 : 0.000874010984586
# year 1  vs  8 : 0.0873333450301
# year 1  vs  9 : 0.00013761848733
# year 1  vs  10 : 3.99692138851e-05
 
# year 2  vs  2 : 1.0
# year 2  vs  3 : 0.223667437532
# year 2  vs  4 : 0.0412316346192
# year 2  vs  5 : 3.24851205834e-05
# year 2  vs  6 : 0.124351819987
# year 2  vs  7 : 0.00634322132845
# year 2  vs  8 : 0.154375886196
# year 2  vs  9 : 0.00278514547228
# year 2  vs  10 : 0.000534915445594
 
# year 3  vs  3 : 1.0
# year 3  vs  4 : 0.49924269147
# year 3  vs  5 : 0.00339927087022
# year 3  vs  6 : 0.57055632215
# year 3  vs  7 : 0.0760320913733
# year 3  vs  8 : 0.931907991639
# year 3  vs  9 : 0.0381657327876
# year 3  vs  10 : 0.0133309713848
 
# year 4  vs  4 : 1.0
# year 4  vs  5 : 0.128416922032
# year 4  vs  6 : 0.958493488876
# year 4  vs  7 : 0.57653356681
# year 4  vs  8 : 0.663656576336
# year 4  vs  9 : 0.254970067512
# year 4  vs  10 : 0.108221945786
 
# year 5  vs  5 : 1.0
# year 5  vs  6 : 0.175355146916
# year 5  vs  7 : 0.849113900902
# year 5  vs  8 : 0.0272754410378
# year 5  vs  9 : 0.980685059436
# year 5  vs  10 : 0.734218792938
 
# year 6  vs  6 : 1.0
# year 6  vs  7 : 0.659437612892
# year 6  vs  8 : 0.564305279925
# year 6  vs  9 : 0.566571893762
# year 6  vs  10 : 0.220440191632
 
# year 7  vs  7 : 1.0
# year 7  vs  8 : 0.278196294869
# year 7  vs  9 : 0.536192159065
# year 7  vs  10 : 0.279270771516
 
# year 8  vs  8 : 1.0
# year 8  vs  9 : 0.081941597619
# year 8  vs  10 : 0.045927424104
 
# year 9  vs  9 : 1.0
# year 9  vs  10 : 0.606183844246
 
# year 10  vs  10 : 1.0

In [ ]:
# # # # #  p-value of KS test 2-sample PRE, >250 citations group:

# year 0  vs  0 : 1.0
# year 0  vs  1 : 0.51709066564
# year 0  vs  2 : 0.0514679389187
# year 0  vs  3 : 0.00692180847247
# year 0  vs  4 : 0.00710737533515
# year 0  vs  5 : 0.00460542067032
# year 0  vs  6 : 5.40741706755e-05
# year 0  vs  7 : 0.000463542444765
# year 0  vs  8 : 3.31640589664e-05
# year 0  vs  9 : 8.11946494924e-05
# year 0  vs  10 : 2.13264295159e-05
 
# year 1  vs  1 : 1.0
# year 1  vs  2 : 0.000276911963314
# year 1  vs  3 : 5.06503782435e-05
# year 1  vs  4 : 5.09751692474e-05
# year 1  vs  5 : 5.92843733064e-06
# year 1  vs  6 : 5.24717217854e-10
# year 1  vs  7 : 1.65087873865e-09
# year 1  vs  8 : 1.32314213031e-10
# year 1  vs  9 : 2.97862248361e-10
# year 1  vs  10 : 1.95773615977e-13
 
# year 2  vs  2 : 1.0
# year 2  vs  3 : 0.482870450704
# year 2  vs  4 : 0.227547706282
# year 2  vs  5 : 0.0968981616195
# year 2  vs  6 : 0.00025772437401
# year 2  vs  7 : 0.00624652367325
# year 2  vs  8 : 0.000316814345548
# year 2  vs  9 : 0.00288939931427
# year 2  vs  10 : 4.04438640525e-05
 
# year 3  vs  3 : 1.0
# year 3  vs  4 : 0.980138646443
# year 3  vs  5 : 0.497937207215
# year 3  vs  6 : 0.0103700085649
# year 3  vs  7 : 0.10231043863
# year 3  vs  8 : 0.014327743148
# year 3  vs  9 : 0.0693743042052
# year 3  vs  10 : 0.00144759912811
 
# year 4  vs  4 : 1.0
# year 4  vs  5 : 0.802559445596
# year 4  vs  6 : 0.0488576595894
# year 4  vs  7 : 0.222591497844
# year 4  vs  8 : 0.0240301565208
# year 4  vs  9 : 0.0956023816707
# year 4  vs  10 : 0.00127924769725
 
# year 5  vs  5 : 1.0
# year 5  vs  6 : 0.063760172978
# year 5  vs  7 : 0.369198106168
# year 5  vs  8 : 0.0762577321672
# year 5  vs  9 : 0.103908657036
# year 5  vs  10 : 0.00339608560251
 
# year 6  vs  6 : 1.0
# year 6  vs  7 : 0.741201841833
# year 6  vs  8 : 0.723611057391
# year 6  vs  9 : 0.789698810357
# year 6  vs  10 : 0.0779946041634
 
# year 7  vs  7 : 1.0
# year 7  vs  8 : 0.716556243384
# year 7  vs  9 : 0.764406144419
# year 7  vs  10 : 0.122174019579
 
# year 8  vs  8 : 1.0
# year 8  vs  9 : 0.900425722275
# year 8  vs  10 : 0.267012134444
 
# year 9  vs  9 : 1.0
# year 9  vs  10 : 0.164937678702
 
# year 10  vs  10 : 1.0



In [ ]:
# # # # # #  p-value of KS test 2-sample PRE, mid-cited 45-55 citations group:

# year 0  vs  0 : 1.0
# year 0  vs  1 : 0.173736586788
# year 0  vs  2 : 0.00495055763109
# year 0  vs  3 : 0.000624429006997
# year 0  vs  4 : 0.000690046643794
# year 0  vs  5 : 0.000478833940901
# year 0  vs  6 : 1.34447401913e-06
# year 0  vs  7 : 2.81224034467e-07
# year 0  vs  8 : 9.84831065443e-06
# year 0  vs  9 : 1.62761153675e-08
# year 0  vs  10 : 7.3176291294e-08
 
# year 1  vs  1 : 1.0
# year 1  vs  2 : 0.197081074161
# year 1  vs  3 : 0.0150172929202
# year 1  vs  4 : 0.0017386611302
# year 1  vs  5 : 0.00385574332135
# year 1  vs  6 : 4.01746181283e-07
# year 1  vs  7 : 5.35037176585e-08
# year 1  vs  8 : 4.53238800735e-07
# year 1  vs  9 : 4.69057193599e-10
# year 1  vs  10 : 1.20411847247e-09
 
# year 2  vs  2 : 1.0
# year 2  vs  3 : 0.357152730524
# year 2  vs  4 : 0.112940720893
# year 2  vs  5 : 0.354961296283
# year 2  vs  6 : 9.65102433202e-05
# year 2  vs  7 : 0.000241966280135
# year 2  vs  8 : 0.000389146215715
# year 2  vs  9 : 5.80347965289e-06
# year 2  vs  10 : 5.10979091122e-06
 
# year 3  vs  3 : 1.0
# year 3  vs  4 : 0.467607865706
# year 3  vs  5 : 0.929039815894
# year 3  vs  6 : 0.0138493764536
# year 3  vs  7 : 0.00882465622541
# year 3  vs  8 : 0.0106845805468
# year 3  vs  9 : 0.000174330542415
# year 3  vs  10 : 0.000124277117491
 
# year 4  vs  4 : 1.0
# year 4  vs  5 : 0.426819288272
# year 4  vs  6 : 0.00445589549017
# year 4  vs  7 : 0.00854959053356
# year 4  vs  8 : 0.00068009099017
# year 4  vs  9 : 4.19631011382e-05
# year 4  vs  10 : 9.47138241207e-06
 
# year 5  vs  5 : 1.0
# year 5  vs  6 : 0.0147751585864
# year 5  vs  7 : 0.0164978795275
# year 5  vs  8 : 0.0507594032608
# year 5  vs  9 : 0.00174758192767
# year 5  vs  10 : 0.00132947842804
 
# year 6  vs  6 : 1.0
# year 6  vs  7 : 0.890820648709
# year 6  vs  8 : 0.79386859213
# year 6  vs  9 : 0.631030276644
# year 6  vs  10 : 0.315089407451
 
# year 7  vs  7 : 1.0
# year 7  vs  8 : 0.402462641025
# year 7  vs  9 : 0.232022905926
# year 7  vs  10 : 0.259501143441
 
# year 8  vs  8 : 1.0
# year 8  vs  9 : 0.339994369434
# year 8  vs  10 : 0.387501199775
 
# year 9  vs  9 : 1.0
# year 9  vs  10 : 0.988245465301
 
# year 10  vs  10 : 1.0
 


In [ ]:
# # # # # #  p-value of KS test 2-sample PRE, 1-4 citations group:

# year 0  vs  0 : 1.0
# year 0  vs  1 : 0.779324705415
# year 0  vs  2 : 0.449909696512
# year 0  vs  3 : 0.585892142938
# year 0  vs  4 : 0.914255026344
# year 0  vs  5 : 0.147269101293
# year 0  vs  6 : 0.600851233807
# year 0  vs  7 : 0.559066975535
# year 0  vs  8 : 0.487560822256
# year 0  vs  9 : 0.396697114537
# year 0  vs  10 : 0.519916998559
 
# year 1  vs  1 : 1.0
# year 1  vs  2 : 0.464353086552
# year 1  vs  3 : 0.814702074918
# year 1  vs  4 : 0.712065151425
# year 1  vs  5 : 0.424931641219
# year 1  vs  6 : 0.961174786319
# year 1  vs  7 : 0.904028080816
# year 1  vs  8 : 0.821002170201
# year 1  vs  9 : 0.8504974226
# year 1  vs  10 : 0.946156952098
 
# year 2  vs  2 : 1.0
# year 2  vs  3 : 0.867461716305
# year 2  vs  4 : 0.608180422492
# year 2  vs  5 : 0.788572338437
# year 2  vs  6 : 0.953643594811
# year 2  vs  7 : 0.955174245517
# year 2  vs  8 : 0.716603354124
# year 2  vs  9 : 0.90200135416
# year 2  vs  10 : 0.956307934207
 
# year 3  vs  3 : 1.0
# year 3  vs  4 : 0.838789845978
# year 3  vs  5 : 0.478171406848
# year 3  vs  6 : 0.558830508542
# year 3  vs  7 : 0.811075694966
# year 3  vs  8 : 0.393954168423
# year 3  vs  9 : 0.491291571594
# year 3  vs  10 : 0.426055725244
 
# year 4  vs  4 : 1.0
# year 4  vs  5 : 0.29289252297
# year 4  vs  6 : 0.807037371538
# year 4  vs  7 : 0.498175846422
# year 4  vs  8 : 0.676920690355
# year 4  vs  9 : 0.701608347433
# year 4  vs  10 : 0.717749222223
 
# year 5  vs  5 : 1.0
# year 5  vs  6 : 0.814671707811
# year 5  vs  7 : 0.739380519191
# year 5  vs  8 : 0.619699608533
# year 5  vs  9 : 0.985603821487
# year 5  vs  10 : 0.959093518591
 
# year 6  vs  6 : 1.0
# year 6  vs  7 : 0.974294588935
# year 6  vs  8 : 0.895475793795
# year 6  vs  9 : 0.970504614329
# year 6  vs  10 : 0.98494572419
 
# year 7  vs  7 : 1.0
# year 7  vs  8 : 0.999987651055
# year 7  vs  9 : 0.929712855395
# year 7  vs  10 : 0.866516304392
 
# year 8  vs  8 : 1.0
# year 8  vs  9 : 0.982650737704
# year 8  vs  10 : 0.881223018063
 
# year 9  vs  9 : 1.0
# year 9  vs  10 : 0.999555214362
 
# year 10  vs  10 : 1.0

In [ ]:
# # box plotting for citing papers of top/intermediate/low   JOURNALs:



# label_selected = 't'  #  top  or intermediate or low  or random





# offset = 0.
# label_plot_yplus1  ='yes'  # plotin the y axis either number of citations, or number of citations  +1

# opacity= .75

# range_y = [0.9,4 ]

# if label_journal == "PRL":
#     range_y = [.35,4 ]
    
# elif label_journal == "PRE":
#     range_y = [.5,3.6 ]
 







# if label_selected == 't':  
#     dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_top_JOURNAL_info 
#     lista_JOURNAL_to_look_for = list_top_UTs
    
#     offset = 0.01
    
#     color= '#999999'   # grey       #lista_colors[0]
# #     info = ', >='+str(value_top)+'citations'
   
    
# elif label_selected == 'i':  
#     dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_intermediate_JOURNAL_info 
#     lista_JOURNAL_to_look_for = list_intermediate_UTs
    
#     color=  '#7fbf7b' #green     lista_colors[2]
#     opacity= 1.
    
# #     info = linfo = ', '+str(interval_intermediate)+' citations'
    
# elif label_selected == 'l':  
#     dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_low_JOURNAL_info 
#     lista_JOURNAL_to_look_for = list_low_UTs
    
#     color= '#ef8a62'     #  salmon    lista_colors[1]
    
#     #info =  ', '+str(interval_low)+' citations'
    
#     opacity= 1.
# elif label_selected == 'r':      
#     dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_RANDOM_SAMPLE_JOURNAL_info 
#     lista_JOURNAL_to_look_for = list_RANDOM_SAMPLE_UTs
    
#     color=  '#af8dc3'   #  purple    lista_colors[-1]
#     offset = -0.01
    
#     #info =  ', random sample'            
    
    
# else:
#     exit()
    
    
    
    
    
    
# n= str(len(dict_UT_ref_of_selected_JOURNAL_info))    
    
    
    






# master_dict_reset_year_list_citations_of_citing_papers = {}




# for JOURNAL_UT in lista_JOURNAL_to_look_for:   # either i focus on top JOURNAL papers, intermediate or low
    
   

#     JOURNAL_year = master_dict_JOURNAL_UT_info[JOURNAL_UT]['year']
#     #print (JOURNAL_UT, JOURNAL_year)
#     list_citing_papers_of_JOURNAL = master_dict_JOURNAL_UT_info[JOURNAL_UT]['list_cit']


#     for citing_paper in list_citing_papers_of_JOURNAL:

#         try:
#             year = dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['year']
#             num_citations_of_citing = len(dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['list_cit'])

#             reset_year = year - JOURNAL_year

#             try:
#                 master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)
#             except KeyError:
#                 master_dict_reset_year_list_citations_of_citing_papers[reset_year] = []
#                 master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)

#         except: pass

    
        
        
    
# for reset_year in sorted(master_dict_reset_year_list_citations_of_citing_papers):
#     if (reset_year <=10)  and (reset_year >= 0):   ### i only plot the first 10 years
#         print (reset_year, np.median(master_dict_reset_year_list_citations_of_citing_papers[reset_year]),np.mean(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), \
#                2.*np.std(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), len(set(master_dict_reset_year_list_citations_of_citing_papers[reset_year])))

#         title_y='Number citations of citing paper'
#         lista_y = master_dict_reset_year_list_citations_of_citing_papers[reset_year]
#         if label_plot_yplus1 =='yes':
#             lista_y = [i+1 for i in master_dict_reset_year_list_citations_of_citing_papers[reset_year]]
#             title_y='Number citations of citing paper +1'
            
#         trace0 = go.Box(
#                 y=lista_y,  
#                # x=[(reset_year+offset)]*len(lista_y),   #  to fabricate some offset between the different traces
#                 name = str(reset_year),#+' (N:'+str(len(master_dict_reset_year_list_citations_of_citing_papers[reset_year]))+")",
#                 opacity = opacity, 
#                 marker = dict(
#                         color = color,#'#808080',
#                         ),
#             boxmean=True,
#             )

#         data.append(trace0)
        
# #         print (lista_y)
# #         print ([reset_year+offset]*len(lista_y))

        
        
 
# font_gral = 60
   
# layout = Layout(    
#     title = '',#citing papers of '+label_selected+' JOURNALJOURNALfo+"<br>N:"+n,
#     showlegend=False,
 
#     yaxis=dict(
#         title=title_y, 
#         range = range_y,  #[0.1,3.5],   # ojo!!! if long scale, also log range, not linear!!!   #[.5,4.5], 
#         type='log',
#         titlefont=dict(            
#             size=font_gral,
#             color='black'
#         #    color='lightgrey'
#         ),  
#         tickfont=dict(               
#             size=font_gral-15,
#             color='black'
#         ),
#         showgrid=True,
#         gridcolor='#525252',
#         gridwidth=1,
#     ),  
       
    
    
#     xaxis=dict(
#         title='Age difference (citing paper - cited paper)', 
#         titlefont=dict(            
#             size=font_gral,
#             color='black'
#         #    color='lightgrey'
#         ),  
#         tickfont=dict(               
#             size=font_gral -15, 
#             color='black'
#         ),
#         tickvals = [0,1,2,3,4,5,6,7,8,9,10],
#     ),
    
    
    
    
#         margin=go.Margin(
#         l=200,
#        # r=50,
#         b=150,
#         t=20,
#         #pad=80
#     ),

    
    
    
    
    

# )
    

# if label_selected == 't':  


#     fig = Figure(data=data, layout=layout)

#     fig_name='box_plot_'+label_selected+'_'+label_journal
#     path='/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/'
#     offline.plot(fig, auto_open=True, image = 'png', image_filename=fig_name ,
#                  output_type='file',image_width=2000, image_height=1400, filename=path+fig_name+".html", validate=False)



In [ ]:
# landa =  0.02703153
# tau = 1./landa
# half_life = tau* np.log(2) 
    
# print ("half-life:",round(half_life,1), " years")

In [ ]:

# ### REFACTORING CODE
# ### box plots with offset for the different traces, so it is easy to see

# data = []


# include_random = 'no'

# label_plot_yplus1 = 'yes'  #'yes'



# label_log_y = 'log10'  # or log or log10  or no  (ojo! only active if if label_plot_yplus1=1)  ## also comment lines in the layout about range!!



# initial_offset = -0.3
# delta_offset_value = 0.15   # this will also afect the width of the boxes



# range_y = [0.24,2.51]


# # this is for plotting number of citations (not log of numb cit)
# # if label_journal == "PRL":
# #     range_y = [.35,3.3 ]
    
# # elif label_journal == "PRE":
# #     range_y = [.2,3.3 ]
    
# # elif label_journal == "PNAS":
# #     range_y = [.25,3.3 ]



 

# print (label_journal)
    
# # lista_tickvals_y = [2,5,10,20,50,100,200,500,1000 ]    
# lista_tickvals_y = [.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75]   # this is just the labels of the axis, not the range of values themselves!
# # if label_log_y != 'no' :
# #     lista_tickvals_y = [0,1,2,3,4,5,6,7,8,9 ]    



# boxpoints_option = False  #'outliers'   # 'outliers',  #'suspectedoutliers',  #False, #'all',  # False,# THIS DICTATES THE BEHAVIOUR OF THE WHISKERS!
# opacity_points = 0.     # to hide all points but keep the same whiskers, set it to 0.0
# opacity_box = 1.    



# width_line = 7
# width_whisker = 1.     # [0,1] value  (horizontal line of whisker)
# width_line_whiskers = 4.  # vertical line of whisker
    
    
    
# #### for the colors of top, intermediate and low

# color_rand =  '#af8dc3'   #  purple    lista_colors[-1]



# if label_journal == "PNAS":
#     lista_colores = ['#330080','#6600ff','#c299ff']  # purples 
#     color_rand = '#006d2c'
    
# elif label_journal == "PRE":
#     lista_colores =  ['#804000','#ff8c1a','#ffe680']    #    ['#993404','#fe9929','#fee391']   # oranges
    
# elif label_journal == "PRL":
#     lista_colores = ['#08519c', '#6baed6','#c6dbef']   ## blues

# elif label_journal == "SCIENCE":
#     lista_colores = ['#a50f15','#fb6a4a','#fcbba1']   # reds        
        
# elif label_journal == "NATURE":                 
#      lista_colores = ['#006d2c','#66c2a4','#ccece6']      ## greens            

# elif label_journal == "JTB":
#     lista_colores = ['#252525','#636363','#bdbdbd']  # greys




            
# title_y='Number of citations'
# if label_plot_yplus1 =='yes' :            
#     title_y='Number of citations +1'
#     if label_log_y == 'log'  :
        
#         title_y='Ln(Number of citations +1)'
#     if label_log_y == 'log10'  :
        
#         title_y='log10(Number of citations +1)'        
            
            
# font_gral = 60





# lista_traces = ['t','i','l','r']




# #lista_dicts = [dict_UT_ref_of_selected_top_JOURNAL_info,dict_UT_ref_of_selected_intermediate_JOURNAL_info, dict_UT_ref_of_selected_low_JOURNAL_info,\
#               #dict_UT_ref_of_selected_RANDOM_SAMPLE_JOURNAL_info]





# #lista_listas_to_look_for = [list_top_UTs, list_intermediate_UTs, list_low_UTs, list_RANDOM_SAMPLE_UTs] 
    




# offset = initial_offset


# for i in range(len(lista_traces)):
    
#     label_selected = lista_traces[i]
    
#     dict_UT_ref_of_selected_JOURNAL_info = lista_dicts[i]
#     lista_JOURNAL_to_look_for = lista_listas_to_look_for[i]
    
#     if i>len(lista_traces):
#         color_trace = lista_colores[i]
#     else:
#         color_trace = color_rand
               
               

#        ### get the data
#     master_dict_reset_year_list_citations_of_citing_papers = {}
#     for JOURNAL_UT in lista_JOURNAL_to_look_for:   # either i focus on top JOURNAL papers, intermediate or low     

#         JOURNAL_year = master_dict_JOURNAL_UT_info[JOURNAL_UT]['year']

#         list_citing_papers_of_JOURNAL = master_dict_JOURNAL_UT_info[JOURNAL_UT]['list_cit']


#         for citing_paper in list_citing_papers_of_JOURNAL:

#             try:
#                 year = dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['year']
#                 num_citations_of_citing = len(dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['list_cit'])

#                 reset_year = year - JOURNAL_year

#                 try:
#                     master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)
#                 except KeyError:
#                     master_dict_reset_year_list_citations_of_citing_papers[reset_year] = []
#                     master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)

#             except: pass



#     cont  = 0        
#     lista_val_line_x = []
#     lista_val_line_y = []
#     for reset_year in sorted(master_dict_reset_year_list_citations_of_citing_papers):
#         if (reset_year <=10)  and (reset_year >= 0):   ### i only plot the first 10 years
#     #         print (reset_year, np.median(master_dict_reset_year_list_citations_of_citing_papers[reset_year]),np.mean(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), \
#     #                2.*np.std(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), len(set(master_dict_reset_year_list_citations_of_citing_papers[reset_year])))



#             lista_y = master_dict_reset_year_list_citations_of_citing_papers[reset_year]
#             if label_plot_yplus1 =='yes':
#                 lista_y = [i+1 for i in master_dict_reset_year_list_citations_of_citing_papers[reset_year]]

#                 if label_log_y == 'log':
#                     lista_y = [np.log(i) for i in lista_y]
#                 elif label_log_y == 'log10':
#                     lista_y = [np.log10(i) for i in lista_y]


#             lista_x = [(reset_year+offset)]* len(lista_y)



#             lista_val_line_y.append(np.median(lista_y))
#             lista_val_line_x.append(reset_year+offset)

#             flag_show_legend = False
#             if cont ==0:
#                 flag_show_legend = True




#             trace0 = go.Box(
#                     y = lista_y,  
#                     x = lista_x,                
#     #                 opacity = opacity_box, 
#                     whiskerwidth= 0,#width_whisker ,
#                     marker = dict(
#                                 color = color_trace,#color,#'##size = 0,  # this doesnt work!!!
#                                 #opacity = opacity_points, 
#                             ),
#                     boxmean=True,
#                     showlegend=flag_show_legend,
#                     #boxpoints = False,  #boxpoints_option, #'outliers',  #'suspectedoutliers',  #False, #'all',  # False,#
#                     line=dict(width=0)#width_line_whiskers),
#                 )






#             cont  +=1


#             data.append(trace0)


#     #   value_top = 170    
#     #     interval_intermediate=[55,75]    
#     #     interval_low=[5,15] 





#     trace1 = go.Scatter(
#         x = lista_val_line_x,
#         y = lista_val_line_y,
#         mode = 'lines+markers',    
#         line = dict(
#             color = color_trace,#color,
#             width = width_line,),
#         showlegend=False



#     )    
#     data.append(trace1)
               
               
#     offset += delta_offset_value   # ready for the next box trace
#     # print (offset)



#     ## i fit to an exponential decay:
#     #  x = numpy.array([10, 19, 30, 35, 51])
#     # >>> y = numpy.array([1, 7, 20, 50, 79])

#     # scipy.optimize.curve_fit(lambda t,a,b: a*numpy.exp(b*t),  x,  y,  p0=(4, 0.1))  #  p0 is the initial guess for the papram
#     # (array([ 4.88003249,  0.05531256]),
#     #  array([[  1.01261314e+01,  -4.31940132e-02],
#     #         [ -4.31940132e-02,   1.91188656e-04]]))   
#     # y ≈ 4.88 exp(0.0553 x). much better.




#     print ("fit to trace:",label_selected)
#     #results = scipy.optimize.curve_fit(lambda t,A,landa: A*np.exp(-1.*landa*t),  lista_val_line_x,  lista_val_line_y)
#     # print (results)


#     results = scipy.optimize.curve_fit(lambda t,A,landa: A*np.exp(-1.*landa*t),  lista_val_line_x,  lista_val_line_y, p0=(lista_val_line_y[0],0.1))
#     # print (results)


#     A = results[0][0]
#     landa = results[0][1]
#     tau = 1./landa
#     half_life = tau* np.log(2) 

#     print ("A:",A,",   landa:", landa,',   tau:', round(tau,1),"years,     half-life:",round(half_life,1), " years\n")


#     #τ is the time at which the population of the assembly is reduced to 1/e ≈ 0.367879441 times its initial value.
#     ### interpretaion of example results:
#     # (array([ 4.88003249,  0.05531256]),
#     #  array([[  1.01261314e+01,  -4.31940132e-02],
#     #         [ -4.31940132e-02,   1.91188656e-04]]))
#     # # y ≈ 4.88 exp(0.0553 x). much better.


        
        
        
        

# # #################
# # #### TRACE:   THE INTERMEDIATE  values


# # label_selected = 'i'
# # if label_selected == 'i':  
# #     dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_intermediate_JOURNAL_info 
# #     lista_JOURNAL_to_look_for = list_intermediate_UTs
   
# #     offset += delta_offset_value 


# #    ### get the data
# # master_dict_reset_year_list_citations_of_citing_papers = {}
# # for JOURNAL_UT in lista_JOURNAL_to_look_for:   # either i focus on top JOURNAL papers, intermediate or low     

# #     JOURNAL_year = master_dict_JOURNAL_UT_info[JOURNAL_UT]['year']
    
# #     list_citing_papers_of_JOURNAL = master_dict_JOURNAL_UT_info[JOURNAL_UT]['list_cit']


# #     for citing_paper in list_citing_papers_of_JOURNAL:

# #         try:
# #             year = dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['year']
# #             num_citations_of_citing = len(dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['list_cit'])

# #             reset_year = year - JOURNAL_year

# #             try:
# #                 master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)
# #             except KeyError:
# #                 master_dict_reset_year_list_citations_of_citing_papers[reset_year] = []
# #                 master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)

# #         except: pass


        
# # cont = 0
# # lista_val_line_y = []
# # lista_val_line_x = []        
# # for reset_year in sorted(master_dict_reset_year_list_citations_of_citing_papers):
# #     if (reset_year <=10)  and (reset_year >= 0):   ### i only plot the first 10 years
# # #         print (reset_year, np.median(master_dict_reset_year_list_citations_of_citing_papers[reset_year]),np.mean(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), \
# # #                2.*np.std(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), len(set(master_dict_reset_year_list_citations_of_citing_papers[reset_year])))

        
# #         lista_y = master_dict_reset_year_list_citations_of_citing_papers[reset_year]
# #         if label_plot_yplus1 =='yes':
# #             lista_y = [i+1 for i in master_dict_reset_year_list_citations_of_citing_papers[reset_year]]
            
# #             if label_log_y == 'log':
# #                 lista_y = [np.log(i) for i in lista_y]
# #             elif label_log_y == 'log10':
# #                 lista_y = [np.log10(i) for i in lista_y]        
                
                
                
        
# #         lista_x = [(reset_year+offset)]* len(lista_y)

        
# #         lista_val_line_y.append(np.median(lista_y))
# #         lista_val_line_x.append(reset_year+offset)
        
         
# #         trace0 = go.Box(
# #                 y = lista_y,  
# #                 x = lista_x,                
# #                 opacity = opacity_box, 
# #                 whiskerwidth= width_whisker ,
# #                 marker = dict(
# #                             color = lista_colores[1],#color,#'##size = 0,  # this doesnt work!!!
# #                             opacity = opacity_points, 
# #                         ),
# #                 boxmean=True,
# #                 showlegend=False,
# #                 boxpoints = boxpoints_option, #'outliers',  #'suspectedoutliers',  #False, #'all',  # False,#
# #                 line=dict(width=width_line_whiskers),
# #             )

        
# #         if cont ==0:
# #              trace0 = go.Box(
# #                     y = lista_y,  
# #                     x = lista_x, 
# #                      name = str(interval_intermediate[0])+'-'+str(interval_intermediate[-1])+" citations",
# #                     opacity = opacity_box, 
# #                     whiskerwidth= width_whisker ,
# #                     marker = dict(
# #                             color = lista_colores[1],#color,#'#808080',
# #                             opacity = opacity_points, 
# #                            # size = 0,
# #                             ),
# #                     boxmean=True,
# #                     showlegend=True,
# #                     boxpoints = boxpoints_option, #'outliers',  #'suspectedoutliers',  #False, #'all',  # False,#
# #                     line=dict(width=width_line_whiskers),   
# #                 )
        
# #         cont  +=1
        
# #         data.append(trace0)

    
        
# # #   value_top = 170    
# # #     interval_intermediate=[55,75]    
# # #     interval_low=[5,15] 

    
    
# # trace1 = go.Scatter(
# #     x = lista_val_line_x,
# #     y = lista_val_line_y,
# #     mode = 'lines+markers',    
# #     line = dict(
# #         color = lista_colores[1],#color,
# #         width = width_line,),
# #     showlegend=False,        
    
# # )    
# # data.append(trace1)


# # #print (offset)      

        
        
# # print ("fit to trace: medium-cited")
# # #results = scipy.optimize.curve_fit(lambda t,A,landa: A*np.exp(-1.*landa*t),  lista_val_line_x,  lista_val_line_y)
# # # print (results)


# # results = scipy.optimize.curve_fit(lambda t,A,landa: A*np.exp(-1.*landa*t),  lista_val_line_x,  lista_val_line_y, p0=(lista_val_line_y[0],0.1))
# # # print (results)


# # A = results[0][0]
# # landa = results[0][1]
# # tau = 1./landa
# # half_life = tau* np.log(2) 
    
# # print ("A:",A,",   landa:", landa,',   tau:', round(tau,1),"years,     half-life:",round(half_life,1), " years\n")




        

# # #################
# # #### TRACE:   THE LOW  values

   
# # label_selected = 'l'
# # if label_selected == 'l':  
   
# #     dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_low_JOURNAL_info 
# #     lista_JOURNAL_to_look_for = list_low_UTs
 
           
# #     offset += delta_offset_value 


# #    ### get the data
# # master_dict_reset_year_list_citations_of_citing_papers = {}
# # for JOURNAL_UT in lista_JOURNAL_to_look_for:   # either i focus on top JOURNAL papers, intermediate or low     

# #     JOURNAL_year = master_dict_JOURNAL_UT_info[JOURNAL_UT]['year']
    
# #     list_citing_papers_of_JOURNAL = master_dict_JOURNAL_UT_info[JOURNAL_UT]['list_cit']


# #     for citing_paper in list_citing_papers_of_JOURNAL:

# #         try:
# #             year = dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['year']
# #             num_citations_of_citing = len(dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['list_cit'])

# #             reset_year = year - JOURNAL_year

# #             try:
# #                 master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)
# #             except KeyError:
# #                 master_dict_reset_year_list_citations_of_citing_papers[reset_year] = []
# #                 master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)

# #         except: pass


        

        
# # cont = 0
# # lista_val_line_y = []
# # lista_val_line_x = []
# # for reset_year in sorted(master_dict_reset_year_list_citations_of_citing_papers):
# #     if (reset_year <=10)  and (reset_year >= 0):   ### i only plot the first 10 years
# # #         print (reset_year, np.median(master_dict_reset_year_list_citations_of_citing_papers[reset_year]),np.mean(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), \
# # #                2.*np.std(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), len(set(master_dict_reset_year_list_citations_of_citing_papers[reset_year])))

        
# #         lista_y = master_dict_reset_year_list_citations_of_citing_papers[reset_year]
# #         if label_plot_yplus1 =='yes':
# #             lista_y = [i+1 for i in master_dict_reset_year_list_citations_of_citing_papers[reset_year]]
            
# #             if label_log_y == 'log':
# #                 lista_y = [np.log(i) for i in lista_y]
# #             elif label_log_y == 'log10':
# #                 lista_y = [np.log10(i) for i in lista_y]        
       
    
        
# #         lista_x = [(reset_year+offset)]* len(lista_y)
        
# #         lista_val_line_y.append(np.median(lista_y))
# #         lista_val_line_x.append(reset_year+offset)

        
# #         trace0 = go.Box(
# #                  y = lista_y,  
# #                 x = lista_x,                
# #                 opacity = opacity_box, 
# #                 whiskerwidth= width_whisker ,
# #                 marker = dict(
# #                             color = lista_colores[2],#color,#'##size = 0,  # this doesnt work!!!
# #                             opacity = opacity_points, 
# #                         ),
# #                 boxmean=True,
# #                 showlegend=False,
# #                 boxpoints = boxpoints_option, #'outliers',  #'suspectedoutliers',  #False, #'all',  # False,#
# #                 line=dict(width=width_line_whiskers),
          
# #             )

        
        
# #         if cont ==0:  # i only show the legend for one of them in each series
# #              trace0 = go.Box(
# #                     y = lista_y,  
# #                     x = lista_x, 
# #                      name = str(interval_low[0])+'-'+str(interval_low[-1])+" citations",
# #                     opacity = opacity_box, 
# #                     whiskerwidth= width_whisker ,
# #                     marker = dict(
# #                             color = lista_colores[2],#color,#'#808080',
# #                             opacity = opacity_points, 
# #                            # size = 0,
# #                             ),
# #                     boxmean=True,
# #                     showlegend=True,
# #                     boxpoints = boxpoints_option, #'outliers',  #'suspectedoutliers',  #False, #'all',  # False,#
# #                     line=dict(width=width_line_whiskers),   
# #                 )
        
# #         cont  +=1
        
# #         data.append(trace0)



    
    
# # trace1 = go.Scatter(
# #     x = lista_val_line_x,
# #     y = lista_val_line_y,
# #     mode = 'lines+markers',    
# #     line = dict(
# #         color = lista_colores[2],#color,
# #         width = width_line,),
# #     showlegend=False,
  
# # )    
# # data.append(trace1)

# # # print (offset)        

        
# # print ("fit to trace: poorly-cited")
# # #results = scipy.optimize.curve_fit(lambda t,A,landa: A*np.exp(-1.*landa*t),  lista_val_line_x,  lista_val_line_y)
# # # print (results)


# # results = scipy.optimize.curve_fit(lambda t,A,landa: A*np.exp(-1.*landa*t),  lista_val_line_x,  lista_val_line_y, p0=(lista_val_line_y[0],0.1))
# # # print (results)


# # A = results[0][0]
# # landa = results[0][1]
# # tau = 1./landa
# # half_life = tau* np.log(2) 
    
# # print ("A:",A,",   landa:", landa,',   tau:', round(tau,1),"years,     half-life:",round(half_life,1), " years\n")


        
        
 
     

# # #################
# # #### TRACE:   THE RANDOM DATA

# # if include_random == 'yes':

# #     label_selected = 'r'
# #     if label_selected == 'r':  
# #         dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_RANDOM_SAMPLE_JOURNAL_info 
# #         lista_JOURNAL_to_look_for = list_RANDOM_SAMPLE_UTs

        
# #         offset += delta_offset_value 


# #        ### get the data
# #     master_dict_reset_year_list_citations_of_citing_papers = {}
# #     for JOURNAL_UT in lista_JOURNAL_to_look_for:   # either i focus on top JOURNAL papers, intermediate or low     

# #         JOURNAL_year = master_dict_JOURNAL_UT_info[JOURNAL_UT]['year']
        
# #         list_citing_papers_of_JOURNAL = master_dict_JOURNAL_UT_info[JOURNAL_UT]['list_cit']


# #         for citing_paper in list_citing_papers_of_JOURNAL:

# #             try:
# #                 year = dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['year']
# #                 num_citations_of_citing = len(dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['list_cit'])

# #                 reset_year = year - JOURNAL_year

# #                 try:
# #                     master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)
# #                 except KeyError:
# #                     master_dict_reset_year_list_citations_of_citing_papers[reset_year] = []
# #                     master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)

# #             except: pass


# #     cont = 0
# #     lista_val_line_y = []
# #     lista_val_line_x = []        

# #     for reset_year in sorted(master_dict_reset_year_list_citations_of_citing_papers):
# #         if (reset_year <=10)  and (reset_year >= 0):   ### i only plot the first 10 years
# # #             print (reset_year, np.median(master_dict_reset_year_list_citations_of_citing_papers[reset_year]),np.mean(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), \
# # #                    2.*np.std(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), len(set(master_dict_reset_year_list_citations_of_citing_papers[reset_year])))

            
# #             lista_y = master_dict_reset_year_list_citations_of_citing_papers[reset_year]
# #             if label_plot_yplus1 =='yes':
# #                 lista_y = [i+1 for i in master_dict_reset_year_list_citations_of_citing_papers[reset_year]]
                
# #             if label_log_y == 'log':
# #                 lista_y = [np.log(i) for i in lista_y]
# #             elif label_log_y == 'log10':
# #                 lista_y = [np.log10(i) for i in lista_y]        
       
                
                
# #             lista_x = [(reset_year+offset)]* len(lista_y)

# #             lista_val_line_y.append(np.median(lista_y))
# #             lista_val_line_x.append(reset_year+offset)



# #             trace0 = go.Box(
# #                     y = lista_y,  
# #                     x = lista_x,                
# #                     opacity = opacity_box, 
# #                     whiskerwidth= width_whisker ,
# #                     marker = dict(
# #                                 color = color_rand,#color,#'##size = 0,  # this doesnt work!!!
# #                                 opacity = opacity_points, 
# #                             ),
# #                     boxmean=True,
# #                     showlegend=False,
# #                     boxpoints = boxpoints_option, #'outliers',  #'suspectedoutliers',  #False, #'all',  # False,#
# #                     line=dict(width=width_line_whiskers),
# #                 )


# #             if cont ==0:
# #                  trace0 = go.Box(
# #                         y = lista_y,  
# #                         x = lista_x, 
# #                         name = "random sample",
# #                         opacity = opacity_box, 
# #                         whiskerwidth= width_whisker ,
# #                         marker = dict(
# #                             color = color_rand,#color,#'#808080',
# #                             opacity = opacity_points, 
# #                            # size = 0,
# #                             ),
# #                         boxmean=True,
# #                         showlegend=True,
# #                         boxpoints = boxpoints_option, #'outliers',  #'suspectedoutliers',  #False, #'all',  # False,#
# #                         line=dict(width=width_line_whiskers),   

# #                     )

# #             cont  +=1



# #             data.append(trace0)

            



# #     trace1 = go.Scatter(
# #         x = lista_val_line_x,
# #         y = lista_val_line_y,
# #         mode = 'lines+markers',    
# #         line = dict(
# #             color = color_rand,
# #             width = width_line,),
# #         showlegend=False,


# #     )    
# #     data.append(trace1)


# #    # print (offset)


        
           
      
        
        
        
        
            
    
    
    
    
    
   
# layout = Layout(    
#     title = '',#citing papers of '+label_selected+' JOURNALJOURNALfo+"<br>N:"+n,
#     # showlegend=True,  IF I WRITE THIS INSIDE EACH TRACE, I PICK WHICH LENGENDS TO SHOW!
#     legend=dict(x=.7, 
#                 y=.95,
#                font=dict(            
#                     size=font_gral-5,           
#                 ),
#                ),
#     yaxis=dict(
#         title=title_y, 
#         range = range_y,      ###### un-comment this line if i want to plot linear y-axis!!!
#        #type='log',           ###### un-comment this line if i want to plot linear y-axis!!!
#         titlefont=dict(            
#             size=font_gral,
#             color='black'
#         #    color='lightgrey'
#         ),  
#         tickfont=dict(               
#             size=font_gral-15,
#             color='black'
#         ),
#         tickvals = lista_tickvals_y ,
#         showgrid=True,
#         gridcolor='#525252',
#         gridwidth=1,
#     ),  
       
    
    
#     xaxis=dict(
#         title='Age difference', 
#         titlefont=dict(            
#             size=font_gral,
#             color='black'
#         #    color='lightgrey'
#         ),  
#         tickfont=dict(               
#             size=font_gral -15, 
#             color='black'
#         ),
#         tickvals = [0,1,2,3,4,5,6,7,8,9,10],
#     ),
    
    
    
    
#         margin=go.Margin(
#         l=200,
#        # r=50,
#         b=120,
#         t=20,
#         #pad=80
#     ),

#     boxgap=0.01
    
    
    
    

# )
    
    
# fig = Figure(data, layout =  layout)

# fig_name='box_plot_offset_'+label_journal
# path='/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/'


# offline.plot(fig, auto_open=True, image = 'png', image_filename=fig_name ,
#                  output_type='file',image_width=2200, image_height=1800, filename=path+fig_name+".html", validate=False)



# #### PNAS
# # fit to trace: highly-cited
# # A: 79.5775433126 ,   landa: 0.127033119861 ,   tau: 7.9 years,     half-life: 5.5  years

# # fit to trace: medium-cited
# # A: 52.3421128686 ,   landa: 0.0857932567604 ,   tau: 11.7 years,     half-life: 8.1  years

# # fit to trace: poorly-cited
# # A: 23.1535757648 ,   landa: 0.0282811138629 ,   tau: 35.4 years,     half-life: 24.5  years




# # SCIENCE
# # fit to trace: highly-cited
# # A: 73.5566847391 ,   landa: 0.14482791378 ,   tau: 6.9 years,     half-life: 4.8  years

# # fit to trace: medium-cited
# # A: 41.880926933 ,   landa: 0.0645084725871 ,   tau: 15.5 years,     half-life: 10.7  years

# # fit to trace: poorly-cited
# # A: 19.5927563684 ,   landa: -0.000444452670601 ,   tau: -2250.0 years,     half-life: -1559.6  years




# # NATURE
# # fit to trace: highly-cited
# # A: 59.5875784412 ,   landa: 0.0875669423552 ,   tau: 11.4 years,     half-life: 7.9  years

# # fit to trace: medium-cited
# # A: 41.6401832307 ,   landa: 0.0480770365393 ,   tau: 20.8 years,     half-life: 14.4  years

# # fit to trace: poorly-cited
# # A: 21.0361533805 ,   landa: 0.00208731952169 ,   tau: 479.1 years,     half-life: 332.1  years


# # PRE
# # fit to trace: highly-cited
# # A: 42.484635282 ,   landa: 0.104757926239 ,   tau: 9.5 years,     half-life: 6.6  years

# # fit to trace: medium-cited
# # A: 23.7676029139 ,   landa: 0.0834063429733 ,   tau: 12.0 years,     half-life: 8.3  years

# # fit to trace: poorly-cited
# # A: 5.46116383289 ,   landa: -0.0350143554733 ,   tau: -28.6 years,     half-life: -19.8  years




# # PRL
# # fit to trace: highly-cited
# # A: 60.1177575044 ,   landa: 0.292642091292 ,   tau: 3.4 years,     half-life: 2.4  years

# # fit to trace: medium-cited
# # A: 25.9369434312 ,   landa: 0.110985782474 ,   tau: 9.0 years,     half-life: 6.2  years

# # fit to trace: poorly-cited
# # A: 11.837182788 ,   landa: 0.0430482652405 ,   tau: 23.2 years,     half-life: 16.1  years



# # JTB
# # fit to trace: highly-cited
# # A: 36.0663881783 ,   landa: -0.0129775250214 ,   tau: -77.1 years,     half-life: -53.4  years

# # fit to trace: medium-cited
# # A: 28.7488070266 ,   landa: 0.0149116465487 ,   tau: 67.1 years,     half-life: 46.5  years

# # fit to trace: poorly-cited
# # A: 13.0394080734 ,   landa: -0.0284148992609 ,   tau: -35.2 years,     half-life: -24.4  years


In [ ]:
# ### box plots with offset for the different traces, so it is easy to see

# # working example with random data

# data = []


# for i in range(10):
#     y0 = np.random.randn(50)+i
#     x0 = [i]* len(y0)
#     trace0 = go.Box(
#         y=y0,
#         x = x0,
#         marker = dict(
#             color = 'rgb(214, 12, 140)',
#         )
#     )
#     data.append(trace0)
#     print (y0)
#     print (x0)
#     print ("\n")


# offset = .1    
    
# for i in range(10):
#     y0 = np.random.randn(50)+i
#     x0 = [i+offset]* len(y0)
#     trace0 = go.Box(
#         y=y0,
#         x = x0,
#         marker = dict(
#             color = 'rgb(20, 112, 240)',
#         )
#     )
#     data.append(trace0)
#     print (y0)
#     print (x0)
#     print ("\n")

    
# offset = -0.1        
# for i in range(10):
#     y0 = np.random.randn(50)+i
#     x0 = [i+offset]* len(y0)
#     trace0 = go.Box(
#         y=y0,
#         x = x0,
#         marker = dict(
#             color = 'rgb(110, 12, 240)',
#         )
#     )
#     data.append(trace0)
#     print (y0)
#     print (x0)
#     print ("\n")


    
    
    
    
    
    
    
    

# font_gral = 60
   
# layout = Layout(    
#     title = '',#citing papers of '+label_selected+' JOURNALJOURNALfo+"<br>N:"+n,
#     showlegend=False,
 
#     yaxis=dict(
#         title=title_y, 
#         #range = range_y,   
#         #type='log',
#         titlefont=dict(            
#             size=font_gral,
#             color='black'
#         #    color='lightgrey'
#         ),  
#         tickfont=dict(               
#             size=font_gral-15,
#             color='black'
#         ),
#         showgrid=True,
#         gridcolor='#525252',
#         gridwidth=1,
#     ),  
       
    
    
#     xaxis=dict(
#         title='Age difference (citing paper - cited paper)', 
#         titlefont=dict(            
#             size=font_gral,
#             color='black'
#         #    color='lightgrey'
#         ),  
#         tickfont=dict(               
#             size=font_gral -15, 
#             color='black'
#         ),
#         tickvals = [0,1,2,3,4,5,6,7,8,9,10],
#     ),
    
    
    
    
#         margin=go.Margin(
#         l=200,
#        # r=50,
#         b=150,
#         t=20,
#         #pad=80
#     ),

    
    
    
    
    

# )
    
    
# fig = Figure(data, layout =  layout)

# fig_name='box_plot_offset'+label_selected+'_'+label_journal
# path='/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/'


# offline.plot(fig, auto_open=True, image = 'png', image_filename=fig_name ,
#                  output_type='file',image_width=2000, image_height=1400, filename=path+fig_name+".html", validate=False)

In [ ]:

# fig = Figure(data=data, layout=layout)

# fig_name='box_plot_'+label_selected+'_'+label_journal
# path='/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/'
# offline.plot(fig, auto_open=True, image = 'png', image_filename=fig_name ,
#              output_type='file',image_width=2000, image_height=1400, filename=path+fig_name+".html", validate=False)


In [ ]:

trace1 = {
  "y": [1, 2, 3, 2, 1, 10], 
  "name": "default", 
  "type": "box"
}
trace2 = {
  "y": [1, 2, 3, 2, 1, 10], 
  "boxpoints": False, 
  "name": "no outliers", 
  "type": "box"
}
trace3 = {
  "y": [1, 2, 3, 2, 1, 10], 
  "boxpoints": "all", 
  "name": "jitter boxpoints", 
  "type": "box"
}
data = Data([trace1, trace2, trace3])
layout = {"title": "Comparing boxplot \"boxpoints\" settings"}
fig = Figure(data=data, layout=layout)
offline.plot(fig)

In [ ]:
# # violin plots:  NOT A GOOD IDEA BECAUSE DATA TOO SKEWED


# label_selected = 'top'  #  top  or intermediate or low


# if label_selected == 'top':  
#     dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_top_JOURNAL_info 
#     lista_JOURNAL_to_look_for = list_top_UTs
#     info = ', >='+str(value_top)+'citations'
   
    
# elif label_selected == 'intermediate':  
#     dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_intermediate_JOURNAL_info 
#     lista_JOURNAL_to_look_for = list_intermediate_UTs
#     info = linfo = ', '+str(interval_intermediate)+' citations'
    
# elif label_selected == 'low':  
#     dict_UT_ref_of_selected_JOURNAL_info = dict_UT_ref_of_selected_low_JOURNAL_info 
#     lista_JOURNAL_to_look_for = list_low_UTs
#     info =  ', '+str(interval_low)+' citations'
    
    
    
# n= str(len(dict_UT_ref_of_selected_JOURNAL_info))    
    
    
    






# master_dict_reset_year_list_citations_of_citing_papers = {}




# for JOURNAL_UT in lista_JOURNAL_to_look_for:   # either i focus on top JOURNAL papers, intermediate or low
#     JOURNAL_year = master_dict_JOURNAL_UT_info[JOURNAL_UT]['year']
#     #print (JOURNAL_UT, JOURNAL_year)
#     list_citing_papers_of_JOURNAL = master_dict_JOURNAL_UT_info[JOURNAL_UT]['list_cit']
    
    
#     for citing_paper in list_citing_papers_of_JOURNAL:
        
#         try:
#             year = dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['year']
#             num_citations_of_citing = len(dict_UT_ref_of_selected_JOURNAL_info[citing_paper]['list_cit'])

            
#             reset_year = year - JOURNAL_year
            
#             try:
#                 master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)
#             except KeyError:
#                 master_dict_reset_year_list_citations_of_citing_papers[reset_year] = []
#                 master_dict_reset_year_list_citations_of_citing_papers[reset_year].append(num_citations_of_citing)

#         except: pass
        
# data = []        
# for reset_year in sorted(master_dict_reset_year_list_citations_of_citing_papers):
#     if (reset_year <=10)  and (reset_year >= 0):   ### i only plot the first 10 years
#     # if (reset_year >= 0):  
#         print (reset_year, np.median(master_dict_reset_year_list_citations_of_citing_papers[reset_year]),np.mean(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), 2.*np.std(master_dict_reset_year_list_citations_of_citing_papers[reset_year]), len(set(master_dict_reset_year_list_citations_of_citing_papers[reset_year])))

    
    
    
#         trace = {
#                 "type": 'violin',
#                 "x": reset_year,    
#                 "y": master_dict_reset_year_list_citations_of_citing_papers[reset_year],  
#                 "name": reset_year,
#                 "box": {
#                     "visible": True
#                 },
#             "line": { 
#                 "color": 'black'},
#              "meanline": {
#                     "visible": True
#                 },
#              "fillcolor": '#b3ffcc',
#              #"opacity": 0.6,
#             }
#         data.append(trace)
        
        
   

        
# fig = {
#     "data": data,
#     "layout" : {
#         'title': 'citing papers of '+label_selected+' JOURNAL'+info+"<br>N:"+n,
#         'showlegend':False,
#         "yaxis": {
#             "zeroline": False,
#             'title':'Number citations of citing paper (as of 2017)',
#             'range' : [0,1000],
#            # 'type':'log',
#         },
#         "xaxis": {           
#             'title':'Age difference (citing paper - JOURNAL paper)', 
#         }
#     }
# }


# #iplot(fig, filename='violin/multiple', validate = False)
# offline.plot(fig, auto_open=True, image = 'png', image_filename="histogram_num_cit_JOURNAL_1990_1995.html" ,
#              output_type='file',image_width=2000, image_height=1400, filename=path+fig_name+".html", validate=False)

In [ ]:
#cursor = papers_con.collection.find().sort([("issue.PY",-1)]).limit(10) # i look for the 10 youngest papers (write +1 instead of -1 for the oldest)
# for s in cursor:
#     print(s)
#     print('--------------')

In [ ]:
# ### plot histograms for number of citations and number of references of the selected  JOURNAL


# lista_years =[1990,1991,1992,1993,1994,1995]

# lista_num_cit = []
# lista_num_ref = []
# for UT in master_dict_JOURNAL_UT_info:
#     lista_num_cit.append(len(master_dict_JOURNAL_UT_info[UT]['list_cit']))
#     lista_num_ref.append(master_dict_JOURNAL_UT_info[UT]['num_ref'])



# trace1= Histogram(x=lista_num_cit, histnorm='probability')
# #         x=x1, 
# #      marker=dict(
# #         color='#17BECF',  #  'purple'     #EB89B5',
# #     ),
# #        # name='high cit',
# #       # histnorm='probability',
# #     #cumulative=dict(enabled=True)
# # )




# ##   xbins.start, xbins.end and xbins.size



# ##   xbins.start, xbins.end and xbins.size


# layout = Layout(    
#     title = label_journal+" "+str(lista_years),
#      xaxis=dict(       
#          title= 'Number of citations received as of 2017' , ),
#      yaxis=dict(       
#          title= 'PDF' ,
#       type='log',),
# )




# # layout = Layout(    

# #     bargap=0.2,



# #     xaxis=dict(
        
# #         title= 'Year' , #v1_string.replace("_"," "),   
# #         titlefont=dict(           
# #             size=font_axes,
# #             color='black'
# #         #    color='lightgrey'
# #         ),  
# #         tickfont=dict(               
# #             size=font_ticks,
# #             color='black'
# #         ),
# #         #type='log'
# #     ),
# #     yaxis=dict(
# #         title='Number papers published',    
# #         type='log',
# #         titlefont=dict(            
# #             size=font_axes,
# #             color='black'
# #         #    color='lightgrey'
# #         ),  
# #         tickfont=dict(               
# #             size=font_ticks,
# #             color='black'
# #         ),
        
# #     ),                

# #     margin=go.Margin(
# #         l=250,
# #        # r=50,
# #         b=150,
# #         #t=100,
# #         #pad=80
# #     ),


    
    
   
    
    
    
    
    

# #     bargroupgap=0.1
# #)       

# data = [trace1]#, trace2]#, trace3]

# fig = Figure(data=data, layout=layout)

# fig_name='histogram_num_cit_PNAS_1990_1995'
# path='/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/'

# #iplot(fig, filename=fig_name)


# offline.plot(fig, auto_open=True, image = 'png', image_filename="histogram_num_cit_PNAS_1990_1995.html" ,
#              output_type='file',image_width=2000, image_height=1400, filename=path+fig_name+".html", validate=False)
